In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import re

In [2]:
def comprobar_resultados(url, browser, urls_menos_3000, urls_procesadas):
    urls_a_procesar = [url]
    
    # El bucle while continúa ejecutándose mientras urls_a_procesar no esté vacía. 
    # En cada iteración del bucle, el método .pop() elimina y devuelve la última URL de la lista, asignándola a url_actual.
    while urls_a_procesar:
        url_actual = urls_a_procesar.pop()
        
        # Si la URL actual ya ha sido procesada, se omite y se pasa a la siguiente iteración del bucle.
        # Si la URL no ha sido procesada, se añade al conjunto urls_procesadas.
        if url_actual in urls_procesadas:
            continue
        urls_procesadas.add(url_actual)
        
        try:
            browser.get(url_actual)
            sleep(2)
            
            # Aceptar cookies
            try:
                browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
                sleep(1)
            except:
                pass
            
            # Buscamos elementos de ubicación de nivel 2 en la página.
            # Si no hay elementos de nivel 2, comprobamos el número total de inmuebles en la página actual. Si es menor de 3000, añadimos la URL a urls_menos_3000.
            location_items = browser.find_elements(by=By.CSS_SELECTOR, value="ul.seo-box__locations-items > li.seo-box__location-item > a.seo-box__location-link--level2")
            
            if not location_items:
                # Si no hay elementos de nivel 2, comprobamos el número total de inmuebles en la página actual.
                try:
                    total_inmuebles_text = browser.find_element(by=By.XPATH, value="//span[@class='total']").text
                    total_inmuebles = int(total_inmuebles_text.replace("(", "").replace(")", "").replace(".", ""))
                    # Si es menor de 3000, añadimos la URL a urls_menos_3000.
                    if total_inmuebles < 3000:
                        urls_menos_3000.append(url_actual)
                except:
                    pass
                continue
            
            # Iteramos sobre cada elemento de ubicación y extraemos la URL y el nombre de la comarca.
            for item in location_items:
                nombre_comarca = item.text.strip()
                url_comarca = item.get_attribute('href')
                
                try:
                    total_inmuebles_text = item.find_element(by=By.XPATH, value=".//span[@class='total']").text
                    total_inmuebles = int(total_inmuebles_text.replace("(", "").replace(")", "").replace(".", ""))
                    
                    # Si el número de inmuebles es mayor de 3000, añadimos la URL de la comarca a urls_a_procesar para procesarla más adelante. 
                    if total_inmuebles > 3000:
                        urls_a_procesar.append(url_comarca)
                    # Si es menor de 3000, corregimos la URL y la añadimos a urls_menos_3000.
                    else:
                        comarca_slug = url_comarca.split("/")[-2]  # Obtener la penúltima parte de la URL
                        url_comarca_corregida = f"https://www.pisos.com/venta/pisos-{comarca_slug}/"  # Corregir
                        urls_menos_3000.append(url_comarca_corregida)  # Guardar la URL
                except Exception as e:
                    continue
        
        except Exception as e:
            print(f"Error al procesar la URL {url_actual}: {e}")

# Inicializar el navegador
browser = webdriver.Chrome()
browser.maximize_window()

# Lista de URLs iniciales
urls = [
   "https://www.pisos.com/viviendas/madrid/",
   "https://www.pisos.com/viviendas/la_rioja/",
   "https://www.pisos.com/viviendas/navarra_nafarroa/",
   "https://www.pisos.com/viviendas/asturias/",
   "https://www.pisos.com/viviendas/cantabria/",
   "https://www.pisos.com/viviendas/murcia/",
   "https://www.pisos.com/viviendas/islas_baleares_illes_balears/",
   "https://www.pisos.com/viviendas/las_palmas/",
   "https://www.pisos.com/viviendas/santa_cruz_de_tenerife/",
   "https://www.pisos.com/viviendas/badajoz/",
   "https://www.pisos.com/viviendas/caceres/",
   "https://www.pisos.com/viviendas/guipuzcoa_gipuzkoa/",
   "https://www.pisos.com/viviendas/vizcaya_bizkaia/",
   "https://www.pisos.com/viviendas/castellon_castello/",
   "https://www.pisos.com/viviendas/valencia/",
   "https://www.pisos.com/viviendas/alicante/",
   "https://www.pisos.com/viviendas/teruel/",
   "https://www.pisos.com/viviendas/huesca/",
   "https://www.pisos.com/viviendas/zaragoza/",
   "https://www.pisos.com/viviendas/girona/",
   "https://www.pisos.com/viviendas/lleida/",
   "https://www.pisos.com/viviendas/barcelona/",
   "https://www.pisos.com/viviendas/lugo/",
   "https://www.pisos.com/viviendas/pontevedra/",
   "https://www.pisos.com/viviendas/ourense/",
   "https://www.pisos.com/viviendas/a_coruna/",
   "https://www.pisos.com/viviendas/guadalajara/",
   "https://www.pisos.com/viviendas/toledo/",
   "https://www.pisos.com/viviendas/cuenca/",
   "https://www.pisos.com/viviendas/albacete/",
   "https://www.pisos.com/viviendas/ciudad_real/",
   "https://www.pisos.com/viviendas/salamanca/",
   "https://www.pisos.com/viviendas/avila/",
   "https://www.pisos.com/viviendas/segovia/",
   "https://www.pisos.com/viviendas/soria/",
   "https://www.pisos.com/viviendas/burgos/",
   "https://www.pisos.com/viviendas/palencia/",
   "https://www.pisos.com/viviendas/valladolid/",
   "https://www.pisos.com/viviendas/leon/",
   "https://www.pisos.com/viviendas/zamora/"
   "https://www.pisos.com/viviendas/alava_araba/",
   "https://www.pisos.com/viviendas/almeria/",
   "https://www.pisos.com/viviendas/cadiz/",
   "https://www.pisos.com/viviendas/castellon_castello/",
   "https://www.pisos.com/viviendas/cordoba/",
   "https://www.pisos.com/viviendas/granada/",
   "https://www.pisos.com/viviendas/huelva/",
   "https://www.pisos.com/viviendas/jaen/",
   "https://www.pisos.com/viviendas/malaga/",
   "https://www.pisos.com/viviendas/sevilla/",
   "https://www.pisos.com/viviendas/tarragona/",
   "https://www.pisos.com/venta/pisos-ceuta/",
   "https://www.pisos.com/venta/pisos-melilla/",
   "https://www.pisos.com/viviendas/islas_baleares_illes_balears/",
   "https://www.pisos.com/venta/pisos-cerdanya_francesa/",
   "https://www.pisos.com/venta/pisos-pais_vasco_frances_iparralde/"
]

urls_menos_3000 = []
urls_procesadas = set()

# Recorrer las URLs iniciales
for url in urls:
    comprobar_resultados(url, browser, urls_menos_3000, urls_procesadas)

browser.quit()

# Imprimir las URLs con menos de 3000 resultados
print("URLs con menos de 3000 resultados:")
for url in urls_menos_3000:
    print(url)

Error al procesar la URL https://www.pisos.com/viviendas/madrid/: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.142)
Stacktrace:
0   chromedriver                        0x00000001047982d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x0000000104790934 cxxbridge1$str$ptr + 2708700
2   chromedriver                        0x00000001042f1f90 cxxbridge1$string$len + 93360
3   chromedriver                        0x00000001042cb964 chromedriver + 129380
4   chromedriver                        0x0000000104360edc cxxbridge1$string$len + 547836
5   chromedriver                        0x0000000104379994 cxxbridge1$string$len + 648884
6   chromedriver                        0x000000010432d060 cxxbridge1$string$len + 335232
7   chromedriver                        0x0000000104760c38 cxxbridge1$str$ptr + 2512864
8   chromedriver                        0x0000000104763f58 cxxbridge1$str$ptr + 25259

In [ ]:
with open('urls_a_scrapear.txt', 'w') as file:
    for url in urls_menos_3000:
        file.write(url + '\n')